In [26]:
import pymongo
import pandas as pd
import numpy as np
import json

from collections import OrderedDict

# Data Exploration

The Fraudulent Transactions dataset (sourced from https://www.kaggle.com/datasets/chitwanmanchanda/fraudulent-transactions-data) contains 6,362,620 transactions with the following features:  
* __step__ - maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (30 days simulation).

* __type__ - CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.

* __amount__ - amount of the transaction in local currency.

* __nameOrig__ - customer who started the transaction

* __oldbalanceOrg__ - initial balance before the transaction

* __newbalanceOrig__ - new balance after the transaction

* __nameDest__ - customer who is the recipient of the transaction

* __oldbalanceDest__ - initial balance recipient before the transaction. Note that there is not information for customers that start with M (Merchants).

* __newbalanceDest__ - new balance recipient after the transaction. Note that there is not information for customers that start with M (Merchants).

* __isFraud__ - This is the transactions made by the fraudulent agents inside the simulation. In this specific dataset the fraudulent behavior of the agents aims to profit by taking control or customers accounts and try to empty the funds by transferring to another account and then cashing out of the system.

* __isFlaggedFraud__ - The business model aims to control massive transfers from one account to another and flags illegal attempts. An illegal attempt in this dataset is an attempt to transfer more than 200.000 in a single transaction.

## Uploading data from MongoDB

### Parsing and cleaning JSON

In [27]:
# Load in Fraud Transaction data
with open("Fraud.json", "r") as file:
    fraud_data_str = file.read()

fraud_data_json = json.loads(fraud_data_str)

print(f"Total transactions: {len(fraud_data_json)}")

Total transactions: 6362620


In [31]:
# Convert numerical fields
for i in range(len(fraud_data_json)):
    for field in ["step", "amount", "oldbalanceOrg", "newbalanceOrig", "oldbalanceDest", "isFraud", "isFlaggedFraud"]:
        fraud_data_json[i][field] = float(fraud_data_json[i][field])

### Alternatively, you can import a csv into mongodb

cd into directory where Fraud.csv is:

mongoimport --type csv -d Fraud -c FraudData --headerline Fraud.csv

### Uploading transaction data to MongoDB

In [33]:
# Create client and load in Fraud database
client = pymongo.MongoClient()
db = client["Fraud"]
fraud_data = db["FraudData"]

# Drop all stored documents
# fraud_data.drop()

In [35]:
# Insert all of the transaction JSONs into the database
# results = fraud_data.insert_many(fraud_data_json)

### Example document for a transaction:

In [36]:
fraud_data.find_one()

{'_id': ObjectId('62579d2e859a15c694d31f1a'),
 'step': 1.0,
 'type': 'PAYMENT',
 'amount': 9839.64,
 'nameOrig': 'C1231006815',
 'oldbalanceOrg': 170136.0,
 'newbalanceOrig': 160296.36,
 'nameDest': 'M1979787155',
 'oldbalanceDest': 0.0,
 'newbalanceDest': '0.0',
 'isFraud': 0.0,
 'isFlaggedFraud': 0.0}

### Total documents:

In [37]:
fraud_data.count_documents({})

6362620

## Data Exploration

### Helper Functions

In [38]:
def cursor_df(cursor: pymongo.command_cursor.CommandCursor) -> pd.DataFrame:
    """
    Convert pymongo results cursor to Pandas DataFrame
    
    :param cursor: Pymongo results cursor
    :results: Pandas DataFrame with results from cursor
    """
    return pd.DataFrame(list(cursor))

### Count of fraudulent transactions:

In [39]:
# Group by if it is fraud, counting the number of transactions
stage_group_fraud = {
    "$group": {
        "_id": "$isFraud", 
        "count": {"$sum": 1}
    }
}

pipeline = [
    stage_group_fraud
]

results = fraud_data.aggregate(pipeline)

cursor_df(results)

,_id,count
0,0.0,6354407
1,1.0,8213


The dataset is very imbalanced with over 99.8% non-fraud transactions.

### Count of each transaction type:

In [40]:
# Group by transaction type, counting the number of transactions
stage_group_trans = {
    "$group": {
        "_id": {
            "transaction_type": "$type"
        },
        "count": {"$sum": 1}
    }
}

stage_project = {
    "$project": {
        "transaction_type": "$_id.transaction_type",
        "count": "$count",
        "_id": 0
    }
}

pipeline = [
    stage_group_trans,
    stage_project
]

results = fraud_data.aggregate(pipeline)

cursor_df(results)

,transaction_type,count
0,CASH_OUT,2237500
1,DEBIT,41432
2,CASH_IN,1399284
3,TRANSFER,532909
4,PAYMENT,2151495


### Fraud count by transaction:

In [41]:
# Group by transaction type, fraud, counting the number of transactions
stage_group_trans = {
    "$group": {
        "_id": {
            "transaction_type": "$type",
            "is_fraud": "$isFraud"
        },
        "count": {"$sum": 1}
    }
}

stage_project = {
    "$project": {
        "transaction_type": "$_id.transaction_type",
        "is_fraud": "$_id.is_fraud",
        "count": "$count",
        "_id": 0
    }
}

stage_sort = {
    "$sort": OrderedDict([("transaction_type", 1), ("is_fraud", 1)])
}

pipeline = [
    stage_group_trans,
    stage_project,
    stage_sort
]

results = fraud_data.aggregate(pipeline)

cursor_df(results)

,transaction_type,is_fraud,count
0,CASH_IN,0.0,1399284
1,CASH_OUT,0.0,2233384
2,CASH_OUT,1.0,4116
3,DEBIT,0.0,41432
4,PAYMENT,0.0,2151495
5,TRANSFER,0.0,528812
6,TRANSFER,1.0,4097


The only transactions that are fraudulent are __CASH_OUT__ and __TRANSFER__ transactions.

### Accounts commiting fraudulent transactions

In [48]:
# Filter only for transactions that are fraud
stage_filter_fraud = {
    "$match": {
        "isFraud": {
            "$eq": 1,
        }
    }
}

stage_project = {
    "$project": {
        "origin_account": "$nameOrig",
        "_id": 0
    }
}
    
pipeline = [
    stage_filter_fraud,
    stage_project
]

results = fraud_data.aggregate(pipeline)

fraud_accounts = cursor_df(results)["origin_account"].to_list()
len(fraud_accounts)

8213

There are no accounts that commit multiple fraudulent transactions, with there being 8,213 total fraud transactions commited by 8,213 accounts.

### Non-fraud transactions commited by fraudulent agents

In [51]:
stage_filter_non_fraud = {
    "$match": {
        "$and": [{
            "$or": [{"nameOrig": {"$in": fraud_accounts}},
                    {"nameDest": {"$in": fraud_accounts}}]
        }, 
            {"isFraud": {"$eq": 0}}]
    }
}
    
pipeline = [
    stage_filter_non_fraud
]

results = fraud_data.aggregate(pipeline)

len(cursor_df(results))

28

There are only 28 non-fraudulent transactions that involve an account that commited a fraudulent transaction.

## PAYMENTS

* No negative payments
* All payments are to merchants
* 2,151,495 payments total
    * 774,245 payments with 0 as old and new balance
        * Change both balance to np.nan?
    * 1,050,371 payments with non-zero old and new balance
        * All amounts - oldbalanceOrg - newbalanceOrig < .2
    * 326,879 payments with non-zero old balance and 0 as new balance
        * All payments where amount > newbalanceOrig 
        * Change newbalanceOrig to negative? New column for declined?

In [89]:
pmt = df.loc[(df["type"] == "PAYMENT")]
pmt.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,merchant_dest,orig_balance_change,dest_balance_change
count,2.151495e+06,2.151495e+06,2.151495e+06,2.151495e+06,0.0,0.0,2151495.0,2151495.0,2151495.0,2.151495e+06,0.0
mean,2.443782e+02,1.305760e+04,6.821683e+04,6.183789e+04,NaN,NaN,0.0,0.0,1.0,6.378937e+03,NaN
std,1.426951e+02,1.255645e+04,1.989911e+05,1.969915e+05,NaN,NaN,0.0,0.0,0.0,9.529503e+03,NaN
min,1.000000e+00,2.000000e-02,0.000000e+00,0.000000e+00,NaN,NaN,0.0,0.0,1.0,0.000000e+00,NaN
25%,1.560000e+02,4.383820e+03,0.000000e+00,0.000000e+00,NaN,NaN,0.0,0.0,1.0,0.000000e+00,NaN
50%,2.490000e+02,9.482190e+03,1.053000e+04,0.000000e+00,NaN,NaN,0.0,0.0,1.0,2.135510e+03,NaN
75%,3.350000e+02,1.756122e+04,6.088300e+04,4.965413e+04,NaN,NaN,0.0,0.0,1.0,9.713065e+03,NaN
max,7.180000e+02,2.386380e+05,4.368662e+07,4.367380e+07,NaN,NaN,0.0,0.0,1.0,1.851225e+05,NaN


In [147]:
print("All payments that have 0 as old and new balance:")
len(pmt.loc[(pmt["oldbalanceOrg"] == 0) & (pmt["newbalanceOrig"] == 0)])

All payments that have 0 as old and new balance


774245

In [150]:
print("All payments that have an old balance, no new balance:")
pmt_no_new_bal = pmt.loc[(pmt["oldbalanceOrg"] > 0) & (pmt["newbalanceOrig"] == 0)]
len(pmt_no_new_bal)

All payments that have an old balance, no new balance:


326879

In [149]:
print("All payments with no new balance and enough original balance to cover amount:")
len(pmt_no_new_bal.loc[pmt_no_new_bal["oldbalanceOrg"] > pmt_no_new_bal["amount"]])

All payments with no new balance and enough original balance to cover amount


0

In [152]:
print("All payments that have both origin balance amounts:")
payment_bal_chg = pmt.loc[(pmt["oldbalanceOrg"] > 0) & (pmt["newbalanceOrig"] > 0)]

len(payment_bal_chg.loc[(abs(pmt["amount"] - pmt["orig_balance_change"])) >= 0])

All payments that have both origin balance amounts:


1050371

In [154]:
print("All payments that have both origin balance amounts greater than .2 difference:")
len(payment_bal_chg.loc[(abs(pmt["amount"] - pmt["orig_balance_change"])) > .2])

All payments that have both origin balance amounts greater than .2 difference:


0